# Explore Sentinel-1 RTC AWS Public Dataset

*This notebook explores Pangeo tooling to efficiently work with a collection of Cloud-Optimized Geotiffs*

https://registry.opendata.aws/sentinel-1-rtc-indigo/

The Sentinel-1 mission is a constellation of C-band Synthetic Aperature Radar (SAR) satellites from the European Space Agency launched since 2014. These satellites collect observations of radar backscatter intensity day or night, regardless of the weather conditions, making them enormously valuable for environmental monitoring. These radar data have been processed from original Ground Range Detected (GRD) scenes into a Radiometrically Terrain Corrected, tiled product suitable for analysis. This product is available over the Contiguous United States (CONUS) since 2017 when Sentinel-1 data became globally available.

In [ ]:
# We'll make use of the following great Python libraries
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import panel as pn
import intake
import numpy as np
import os
import pandas as pd
import rasterio
import rioxarray
import s3fs 
import xarray as xr
hv.extension('bokeh')

## Metadata and data search

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Default is 10 MB limit in the GeoTIFF driver for range request merging.

In [ ]:
# This data is stored as tiles on the military grid system
# This visualization helps you pick a tile of interest
grid = 'https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/_downloads/a5f63bfdd3b923cde925b95de3bd1dbe/SENTINEL1_RTC_CONUS_GRID.geojson'
gf = gpd.read_file(grid)
gf.rename(columns=dict(id='tile'), inplace=True) #"id" reserved as a method name
tiles = gv.tile_sources.OSM()
polygons = gf.hvplot.polygons(geo=True, alpha=0.2, hover_cols=['tile'], legend=False)
tiles * polygons

In [ ]:
# Path layout is as follows:
# s3://sentinel-s1-rtc-indigo/tiles/RTC/1/[MODE]/[MGRS UTM zone]/[MGRS latitude label]/[MGRS Grid Square ID]/[YEAR]/[SATELLITE]_[DATE]_[TILE ID]_[ORBIT DIRECTION]/[ASSET]
zone = 12
latLabel = 'S'
square = 'YJ'
year = '*' #all years
date = '*' #all acquisitions
polarization = 'VV'
s3Path = f's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/{zone}/{latLabel}/{square}/{year}/{date}/Gamma0_{polarization}.tif'
print(s3Path[5:])

In [ ]:
%%time

# Find imagery according to S3 path pattern
s3 = s3fs.S3FileSystem(anon=True)
keys = s3.glob(s3Path[5:]) #strip s3://
print(f'Located {len(keys)} images matching {s3Path}:')
keys[:5]

In [ ]:
# want to ensure these are sorted in chronological order (which S1A and S1B can mess up if going just by an alphabetical sort)
keys.sort(key=lambda x: x[-32:-24])

files = [key[-36:] for key in keys]
satellites = [x[:3] for x in files]
dates = [x[4:12] for x in files]
direction = [x[19:22] for x in files]

In [ ]:
# construct some tidy metadata to facilitate future plotting
df = pd.DataFrame(dict(satellite=satellites,
                       date=dates, #string
                       datetime=pd.to_datetime(dates), #pandas timestamp
                       direction=direction,
                       file=files))
df['dt'] = df.datetime.diff() # NOTE: images do not cover exactly the same area b/c this is swath data
df.head()

In [ ]:
# For example, we might want to quickly visualize the acquisition dates, satellite, and pass direction
# This plot shows we have mostly Ascending pass observations from the Sentinel-1B satellite
df.hvplot.scatter(x='datetime', y='satellite', by='direction', marker='dash', size=200, angle=90)

In [ ]:
# Get information on geospatial metadata and internal overviews
s3Paths = ['s3://' + key for key in keys]
url = s3Paths[0]
print(url)
with rasterio.open(url) as src:
    print(src.profile)  
    overview_factors = [src.overviews(i) for i in src.indexes][0]
    overview_levels = list(range(len(overview_factors)))
    print('Overview levels: ', overview_levels)
    print('Overview factors: ',  overview_factors) 

In [ ]:
#%%time
# Create a VRT that points to all files in separate bands
# Seems to take <30seconds for ~200 Tifs
vrtName = f'stack{zone}{latLabel}{square}.vrt'
#cmd = f'gdalbuildvrt -overwrite -separate -input_file_list s3paths.txt {vrtName}'
#!{cmd}

In [ ]:
# Customize the VRT adding date to metadata
#with rasterio.open(vrtName, 'r+') as src:
#    print(f'adding filenames to {vrtName} band descriptions')
#    src.descriptions = files
    # Or add as a metadata
    #src.meta['date'] = files # doesn't work
    #src.update_tags(date=files) # adds   <Metadata> <MDI key="date">   

## Working with low-resolution overviews

In [ ]:
%%time
# Use rioxarray to quickly load overviews for visualization, takes ~12s
da3 = rioxarray.open_rasterio(vrtName, overview_level=3, chunks=dict(band=1)) 
da3

In [ ]:
print(f'Uncompressed dataset size= {da3.nbytes/1e6} Mb')

In [ ]:
%%time
# NOTE the overviews are only ~100MB, so pull it into memory before plotting for speed 
da3 = da3.compute()

In [ ]:
# Overview browser
da3.hvplot.image(clim=(0,0.4), aspect='equal', frame_width=400, cmap='gray', widget_location='bottom')

In [ ]:
# get filepths reading the vrt with rasterio
with rasterio.open(vrtName) as src:
    filenames = src.descriptions
    datetimes = [pd.to_datetime(x[4:12]) for x in filenames]
    
datetimes[:5]

In [ ]:
# add new coordinate to existing dimension 
da = da3.assign_coords(time=('band', datetimes))
# make 'time' active coordinate instead of integer band
da = da.swap_dims({'band':'time'})
# Name the dataset (helpful for hvplot calls later on)
da.name = 'Gamma0VV'

In [ ]:
%%time
# Now we can get montly mosaics pretty easily (mean, min, max, etc)
mosaic_mean = da.where(da!=0).resample(time='1m').mean()

In [ ]:
video = mosaic_mean.hvplot.image(x='x',y='y', clim=(0,0.4), aspect='equal', frame_width=600, cmap='gray', 
                                 widget_type='scrubber', widget_location='bottom') 
video

In [ ]:
# Save our derived dataset to a local file for future use
#mosaic_mean.to_netcdf('mosasic_mean.nc')
#da = xr.open_dataset('mosaic_mean.nc')

In [ ]:
# vector file defining area of interest in lat/lon from geojson.io
gf = gpd.read_file('grand-mesa.geojson')
tiles = gv.tile_sources.EsriUSATopo
bbox = gf.hvplot.polygons(alpha=0.2, geo=True)
tiles * bbox 

In [ ]:
gf.bounds

In [ ]:
gfp = gf.to_crs(da.rio.crs)
print(f' Area of bounding box (km^2): {gfp.area.values[0]*1e-6 :.2f}')
gfp.bounds

In [ ]:
%%time 
# Time series plot (flat area of grand mesa)
# https://hvplot.holoviz.org/user_guide/Timeseries_Data.html

xmin, ymax, xmax, ymin = gfp.bounds.values[0]

daT = da.sel(x=slice(xmin, xmax), 
             y=slice(ymin, ymax))


from bokeh.models import DatetimeTickFormatter
formatter = DatetimeTickFormatter(months='%b %Y')

all_points = daT.where(daT!=0).hvplot.scatter('time', groupby=[], dynspread=True, datashade=True) 
mean_trend = daT.where(daT!=0, drop=True).mean(dim=['x','y']).hvplot.line(title='North Grand Mesa', color='red')
(all_points * mean_trend).opts(xformatter=formatter)

In [ ]:
# save timeseries shown in plot to csv
# scatter() and line() plots have a 'data' attribute that is just a pandas dataframe!
mean_trend.data.to_csv('mean_trend.csv')

## Full Resolution data

In [ ]:
%%time
#To work with the full_resolution dataset, simply do not ask for overview_level
da = rioxarray.open_rasterio(vrtName, masked=True, chunks=dict(band=1))#, x=512, y=512)) 
da = da.assign_coords(time=('band', datetimes))
da = da.swap_dims({'band':'time'})
da.name = 'Gamma0VV'
#da

In [ ]:
%%time 
# Rasterize=True requires holoviews!=1.13.4
# https://github.com/holoviz/holoviews/issues/4653
file_names = da.attrs.pop('long_name') #holoviews doesn't like these names in attributes
images = da.hvplot.image(x='x',y='y',rasterize=True, # rasterize=True is key for large images 
                clim=(0,0.4), aspect='equal', 
                frame_width=400, cmap='gray',
                #widgets={'band': pn.widgets.IntSlider}, #override defaul DiscreteSlider
                widgets={'time': pn.widgets.Select}, #override defaul DiscreteSlider
                widget_location='bottom')
images

In [ ]:
# It's common to reduce the dimensions of the data you're analyzing in some way
# For example, hone in on a subset and compute the monthly mean:
subset = da.sel(x=slice(7.345e5, 7.614e5), 
                y=slice(4.3336e6, 4.306e6)
               ).resample(time='1m').mean() # monthly mean of subset (need time index)

print('dataset size (MB):', subset.nbytes/1e6)
subset

In [ ]:
%%time
# video playback will be smoother if pixel in local or distributed memory
# Takes ~1min to complete
# TODO: figure out dask settings to speed this up
from dask.diagnostics import ProgressBar

with ProgressBar():
    frames = subset.compute()

In [ ]:
video = frames.hvplot.image(x='x',y='y', clim=(0,0.4), aspect='equal', frame_width=600, cmap='gray', 
                                 widget_type='scrubber', widget_location='bottom') 
video

In [ ]:
## Save our derived dataset to a local file for future use
#frames.to_netcdf('full_res_mosaic_mean.nc')
#da = xr.open_dataset('full_res_mosaic_mean.nc')